<a href="https://colab.research.google.com/github/Sarvesh-Kesharwani/Deep-Learning/blob/main/GooglenetV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q ivy

!git clone https://github.com/unifyai/models.git
!cd models && python3 -m pip install -q -e .

!python3 -m pip install torchvision

exit()

fatal: destination path 'models' already exists and is not an empty directory.
  Preparing metadata (setup.py) ... done


In [3]:
import ivy
import torch
import ivy_models
from torchvision import transforms
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import tensorflow as tf

# Inception model implementation

In [4]:

# Building the initial Convolutional Block
class ConvBlock(ivy.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        super(ConvBlock, self).__init__()

    def _build(self, *args, **kwargs):
        self.conv = ivy.Conv2D(self.in_channels, self.out_channels, self.kernel_size, self.stride, self.padding, with_bias=False)
        self.bn = ivy.BatchNorm2D(self.out_channels)
        self.activation = ivy.ReLU()

    def _forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.activation(x)
        return x

In [5]:

class Inception(ivy.Module):
    def __init__(
        self,
        in_channels,
        num1x1,
        num3x3_reduce,
        num3x3,
        num5x5_reduce,
        num5x5,
        pool_proj,
    ):
        self.in_channels = in_channels
        self.num1x1 = num1x1
        self.num3x3_reduce = num3x3_reduce
        self.num3x3 = num3x3
        self.num5x5_reduce = num5x5_reduce
        self.num5x5 = num5x5
        self.pool_proj = pool_proj
        super(Inception, self).__init__()

    def _build(self, *args, **kwargs):
        self.block1 = ivy.Sequential(ConvBlock(self.in_channels, self.num1x1, kernel_size=[1, 1], stride=1, padding=0)
                                     )
        self.block2 = ivy.Sequential(ConvBlock(self.in_channels, self.num3x3_reduce, kernel_size=[1, 1], stride=1, padding=0),
                                    ConvBlock(self.num3x3_reduce, self.num3x3, kernel_size=[3, 3], stride=1, padding=1)
                                    )
        self.block3 = ivy.Sequential(ConvBlock(self.in_channels, self.num5x5_reduce, kernel_size=[1, 1], stride=1, padding=0),
                                    ConvBlock(self.num5x5_reduce, self.num5x5, kernel_size=[3, 3], stride=1, padding=1)
                                    )
        self.block4 = ivy.Sequential(ivy.MaxPool2D(3, 1, 1),
                                    ConvBlock(self.in_channels, self.pool_proj, kernel_size=[1, 1], stride=1, padding=0)
                                    )

    def _forward(self, x):
        block1 = self.block1(x)
        block2 = self.block2(x)
        block3 = self.block3(x)
        block4 = self.block4(x)

        return ivy.concat([block1, block2, block3, block4], axis=3)

In [6]:

class Auxiliary(ivy.Module):
    def __init__(self, in_channels, num_classes):
        self.in_channels = in_channels
        self.num_classes = num_classes
        super(Auxiliary, self).__init__()

    def _build(self, *args, **kwargs):
        self.pool = ivy.AvgPool2D((5, 5), 3, 0)  # ivy.Shape(1, 4, 4, 512)
        self.bn = ivy.BatchNorm2D(128)
        self.conv = ivy.Conv2D(self.in_channels, 128, [1,1], 1, 0, with_bias=False)
        self.activation = ivy.ReLU()
        self.fc1 = ivy.Linear(2048, 1024)
        self.dropout = ivy.Dropout(0.7)
        self.fc2 = ivy.Linear(1024, self.num_classes)

    def _forward(self, x):
        out = self.pool(x)
        out = self.conv(out) # contains weights
        out = self.activation(out)
        out = ivy.flatten(out, start_dim=1)
        out = self.fc1(out) # contains weights
        out = self.activation(out)
        out = self.dropout(out)
        out = self.fc2(out) # contains weights
        return out

In [7]:

class GoogLeNet(ivy.Module):
    def __init__(self, num_classes=1000, v: ivy.Container = None,):
        if v is not None:
            self.v = v
        self.num_classes = num_classes
        super(GoogLeNet, self).__init__(v=v)

    def _build(self, *args, **kwargs):
        self.conv1 = ConvBlock(3, 64, [7,7], 2, 3)
        self.pool1 = ivy.MaxPool2D([3,3], 2, 1)
        self.conv2 = ConvBlock(64, 64, [1,1], 1, 0,)
        self.conv3 = ConvBlock(64, 192, [3,3], 1, 1)
        self.pool3 = ivy.MaxPool2D(3, 2, 1)
        self.inception3A = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3B = Inception(256, 128, 128, 192, 32, 96, 64)
        self.pool4 = ivy.MaxPool2D(3, 2, 1)

        self.inception4A = Inception(480, 192, 96, 208, 16, 48, 64)

        # ivy.flatten()
        self.aux4A = Auxiliary(512, self.num_classes)

        self.inception4B = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4C = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4D = Inception(512, 112, 144, 288, 32, 64, 64)

        self.aux4D = Auxiliary(528, self.num_classes)

        self.inception4E = Inception(528, 256, 160, 320, 32, 128, 128)
        self.pool5 = ivy.MaxPool2D(3, 2, 1)

        self.inception5A = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5B = Inception(832, 384, 192, 384, 48, 128,128)
        self.pool6 = ivy.AvgPool2D((7,7), 1, 0) # ((1, 1))

        # ivy.flatten()
        self.dropout = ivy.Dropout(0.4)
        self.fc = ivy.Linear(1024, self.num_classes)


    def _forward(self, x):
        out = self.conv1(x)
        out = self.pool1(out)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.pool3(out)
        out = self.inception3A(out)
        out = self.inception3B(out)
        out = self.pool4(out)
        out = self.inception4A(out)

        aux1 = self.aux4A(out)

        out = self.inception4B(out)
        out = self.inception4C(out)
        out = self.inception4D(out)

        aux2 = self.aux4D(out)

        out = self.inception4E(out)
        out = self.pool5(out)
        out = self.inception5A(out)
        out = self.inception5B(out)
        out = self.pool6(out)
        out = ivy.flatten(out, start_dim=1)
        out = self.dropout(out)
        out = self.fc(out)

        return out, aux1, aux2


# Setting up ivy compiler api key


In [8]:
!mkdir .ivy
!echo -n gDNLm8VEO7QleBN5GdTcfMa7OHWBOWcyOiHWr2KI654= > .ivy/key.pem

mkdir: cannot create directory ‘.ivy’: File exists


# Ivy

In [11]:
def preprocess_image_for_ivy_model(image):
    def load_and_preprocess_img(img, new_size, crop, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
        compose = transforms.Compose(
            [
                transforms.Resize(new_size),
                transforms.CenterCrop(crop),
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std),
            ]
        )
        img = compose(img)
        img = img.unsqueeze(0).permute((0, 2, 3, 1))
        return img.numpy()

    import ivy
    ivy.set_backend("torch")
    img = ivy.asarray(load_and_preprocess_img(image, 256, 224))

    return img


# from PIL import Image
# image = Image.open("/content/cat.jpg")
# preprocessed_image = preprocess_image_for_ivy_model(image)

# display(preprocessed_image)

In [12]:
import torch

compilation_count = 1
ivy_model=None
def infer_1image_using_ivy_model(image):
    def _inceptionNet_torch_weights_mapping(old_key, new_key):
        W_KEY = ["conv/weight"]
        new_mapping = new_key
        if any([kc in old_key for kc in W_KEY]):
            new_mapping = {"key_chain": new_key, "pattern": "b c h w -> h w c b"}
        return new_mapping

    def inceptionNet_v1(pretrained=True):
        """InceptionNet-V1 model"""
        if not pretrained:
            return GoogLeNet()

        reference_model = GoogLeNet()
        url = "https://download.pytorch.org/models/googlenet-1378be20.pth"
        w_clean = ivy_models.helpers.load_torch_weights(
            url,
            reference_model,
            raw_keys_to_prune=["num_batches_tracked"],
            custom_mapping=_inceptionNet_torch_weights_mapping,
            )
        display("calling model with weights!")
        return GoogLeNet(v=w_clean)


    global compilation_count
    global ivy_model
    if compilation_count == 1:
      compilation_count -= 1
      ivy_model = inceptionNet_v1(pretrained=True)
      ivy.set_backend('torch')
      ivy_model.compile(args=(image,))


    output, _, _ = ivy_model(image)
    return output


# Torch

In [13]:
def preprocess_image_for_torch_model(image):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])
    # torch_img = Image.open("/content/cat.jpg")
    torch_img = preprocess(image)
    torch_img = torch.unsqueeze(torch_img, 0)

    return torch_img


In [14]:
# Convert to numpy
img = torch_img.permute((0, 2, 3, 1)).detach().numpy()

NameError: ignored

In [15]:
import torchvision.models as models

def torch_googlenet(pretrained=True):
        """InceptionNet-V1 model"""
        if not pretrained:
            return models.googlenet(pretrained=False)

        reference_model = models.googlenet()
        url = "https://download.pytorch.org/models/googlenet-1378be20.pth"
        w_clean = torch.hub.load_state_dict_from_url(url, progress=True)
        model = models.googlenet()
        model.load_state_dict(w_clean)
        return model

# compiling model one time
torch_model = torch_googlenet()
torch_model.eval()

def infer_1image_using_torch_model(image):
    with torch.no_grad():
        output_gt = torch_model(image)
    return output_gt


# Accuracy

In [16]:
import os
import random
import numpy as np
from PIL import Image

def return_list_of_N_random_images(N):
    def load_images_from_folder(folder_path, num_images=100):
        image_paths = []
        for root, _, files in os.walk(folder_path):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    image_paths.append(os.path.join(root, file))

        if len(image_paths) == 0:
            print("No images found in the provided folder.")
            return None

        if num_images > len(image_paths):
            num_images = len(image_paths)

        random.shuffle(image_paths)
        selected_image_paths = image_paths[:num_images]

        images = []
        for image_path in selected_image_paths:
            try:
                img = Image.open(image_path)
#                 img_array = np.array(img)
                images.append(img)
#                 img.close()
            except Exception as e:
                print(f"Error loading image: {image_path}. Skipping.\nError: {e}")

        return images

    folder_path = "/content/images"
    num_random_images = N
    random_images = load_images_from_folder(folder_path, num_random_images)
    # Now you have a list 'random_images_as_arrays' containing 100 randomly selected images as numpy arrays
    return random_images

In [32]:
def find_model_acc(output_logit, output_gt_logit):
    accuracy = tf.keras.metrics.Accuracy()
    model_acc = torch.tensor([])

    for _ in range(0, 1):
        output_logit = torch.where(output_logit[0] > 0.5, torch.tensor(1), torch.tensor(0))
        output_gt_logit = torch.where(output_gt_logit[0] > 0.5, torch.tensor(1), torch.tensor(0))

        accuracy.update_state(output_logit, output_gt_logit)
        acc_value = accuracy.result().numpy()
        model_acc = torch.cat((model_acc, torch.tensor([acc_value])))

    return model_acc


In [18]:
def run_2inferences_for_each_image_from(no_of_images=1):
    image_sequence=return_list_of_N_random_images(no_of_images)
    for image in image_sequence:
        display(image.size)
        image1 = preprocess_image_for_ivy_model(image)
        output_logit = infer_1image_using_ivy_model(image1)

        image2 = preprocess_image_for_torch_model(image)
        output_gt_logit = infer_1image_using_torch_model(image2)

        model_acc = find_model_acc(output_logit, output_gt_logit)
    display(model_acc)

In [33]:
# %%timeit
run_2inferences_for_each_image_from(1)

(1918, 1280)

tensor([0.7500])